In [1]:
import numpy as np
import numpy as np
import matplotlib.pyplot as plt
from scipy import interpolate
import scipy.optimize as opt
import scipy 
from scipy.optimize import fminbound

# to print plots inline
%matplotlib inline

In [2]:
#parameters
beta = 0.95 #time preference
alpha = 0.5 #This measures the intensity of habit formation and also denotes the nonseparability of preferences over time
Y = 100 #exogenous income
R = 1 #interest rate

#grids
a_lb = 0.1
a_ub = 4
size_a = 200
size_c = 200
a_grid = np.linspace(a_lb, a_ub, size_a)
c_grid = np.linspace(a_lb, a_ub, size_c)

In [3]:
def utility(a, aprime, c_1, alpha, R, Y):
    """
    Per period utility function
    """
    c = Y + (1 + R) * a - aprime
    if c < c_1:
        u = np.log(1)
    else:
        u = np.log(c - alpha * c_1)
                
                   
    return u

In [4]:
def bellmanoperator(V, a_grid, c_grid, params):
    alpha, R, Y, beta = params
    V_func = interpolate.interp2d(a_grid, c_grid, V, kind='cubic')

    TV = np.empty_like(V)
    opta = np.empty_like(TV)

    # : max_a' { u(c,c_1) + beta V(a', c)} #but c = f(a, aprime)
    #This is Equation 2 
    for i, a in enumerate(a_grid):
        def objective(aprime):
            return - utility(a, aprime, c_1, alpha, R, Y) - (beta * V_func(aprime, c))
        aprime_star = fminbound(objective, 1e-6, a - 1e-6)
        opta[i] = aprime_star
        TV[i] = - objective(aprime_star)
                                 
                
                

    return TV, opta


In [5]:
#value function iteration
VFtol = 1e-5
VFdist = 7.0 
VFmaxiter = 200 
V = np.zeros((size_a, size_c))
Vstore = np.zeros((size_a, size_c, VFmaxiter))
VFiter = 1 
V_params = (alpha, R, Y, beta)
while VFdist > VFtol and VFiter < VFmaxiter:
    for a in range(size_a):
        for c in range(size_c):
            for c_1 in range(size_c):
                Vmat[a, c_1, c] = utility(a, aprime, c_1, alpha, R, Y) + (beta * V_func(aprime, c))  
                
    Vstore[:, VFiter] = V
    TV, opta = bellmanoperator(V, a_grid, c_grid, V_params)
    VFdist = (np.absolute(V - TV)).max()  
    print('iteration ', VFiter, ', distance = ', VFdist)
    V = TV
    VFiter += 1
   

if VFiter < VFmaxiter:
    print('coverged after:', VFiter)
else:
    print('did not converge')

VF = V 

NameError: name 'aprime' is not defined